### Imports

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

ModuleNotFoundError: No module named 'seaborn'

### Reading the data


In [10]:
csv_path = "Data/student-por.xls"
df = pd.read_csv(csv_path)

### First look at data

In [11]:
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649 entries, 0 to 648
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   school      649 non-null    object
 1   sex         649 non-null    object
 2   age         649 non-null    int64 
 3   address     649 non-null    object
 4   famsize     649 non-null    object
 5   Pstatus     649 non-null    object
 6   Medu        649 non-null    int64 
 7   Fedu        649 non-null    int64 
 8   Mjob        649 non-null    object
 9   Fjob        649 non-null    object
 10  reason      649 non-null    object
 11  guardian    649 non-null    object
 12  traveltime  649 non-null    int64 
 13  studytime   649 non-null    int64 
 14  failures    649 non-null    int64 
 15  schoolsup   649 non-null    object
 16  famsup      649 non-null    object
 17  paid        649 non-null    object
 18  activities  649 non-null    object
 19  nursery     649 non-null    object
 20  higher    

### Searching for unusual values

In [13]:
df.isnull().sum()

school        0
sex           0
age           0
address       0
famsize       0
Pstatus       0
Medu          0
Fedu          0
Mjob          0
Fjob          0
reason        0
guardian      0
traveltime    0
studytime     0
failures      0
schoolsup     0
famsup        0
paid          0
activities    0
nursery       0
higher        0
internet      0
romantic      0
famrel        0
freetime      0
goout         0
Dalc          0
Walc          0
health        0
absences      0
G1            0
G2            0
G3            0
dtype: int64

In [14]:
df.duplicated().sum()

np.int64(0)

### Data change


In [16]:
#Creating a new column for the total value that is the sum of all grades
df_values = df[['G1','G2','G3']]
df['total'] = df_values.sum(axis=1)
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,total
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,3,4,1,1,3,4,0,11,11,22
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,3,3,1,1,3,2,9,11,11,31
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,3,2,2,3,3,6,12,13,12,37
3,GP,F,15,U,GT3,T,4,2,health,services,...,2,2,1,1,5,0,14,14,14,42
4,GP,F,16,U,GT3,T,3,3,other,other,...,3,2,1,2,5,0,11,13,13,37


In [19]:
#Creating the filters for the highest and lowest grades
df_filteredH = df.sort_values(by='total',ascending=False)
df_filteredH.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,total
338,GP,F,17,R,LE3,T,3,1,services,other,...,1,2,1,1,3,0,18,19,19,56
636,MS,M,18,U,GT3,T,4,4,teacher,teacher,...,2,4,1,4,2,4,17,18,19,54
606,MS,F,17,U,GT3,T,4,2,teacher,other,...,2,3,3,1,5,0,18,18,18,54
332,GP,F,18,U,GT3,T,2,2,at_home,at_home,...,3,3,1,2,2,0,18,18,18,54
594,MS,F,18,U,GT3,T,4,4,teacher,teacher,...,3,5,1,2,1,0,18,18,18,54


In [20]:
df_filteredL = df.sort_values(by='total')
df_filteredL.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,total
567,MS,M,18,R,GT3,T,3,2,services,other,...,3,1,2,2,5,0,4,0,0,4
603,MS,F,18,R,LE3,A,4,2,teacher,other,...,3,1,1,1,5,0,5,0,0,5
605,MS,F,19,U,GT3,T,1,1,at_home,services,...,5,5,2,3,2,0,5,0,0,5
563,MS,M,17,U,GT3,T,2,2,other,other,...,2,1,2,3,5,0,7,0,0,7
440,MS,M,16,U,GT3,T,1,1,at_home,services,...,4,5,4,5,3,0,7,0,0,7
